In [13]:
# pbp_parse.py
#
# parse JSON, make game structure, convert to CSV.
# JSON 데이터를 읽어와 게임 상황을 구현하고
# gameday line 하나에 맞는 text row를 생성, CSV 파일에 저장한다.

import os, json, regex, csv, sys, traceback, time

import pandas as pd
import numpy as np
from pbp_download import download_relay
from tqdm import tqdm, tqdm_notebook, trange

import datetime, requests
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup

# custom library
from utils import print_progress
from new_pbp_download import get_game_ids, get_game_data

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
def parse_batter_result(text):
    for tup in batter_results:
        if text.find(tup[0]) >= 0:
            return tup[1], tup[2]


def parse_batter_as_runner(text):
    runner = text.split(' ')[0]
    
    result = 'o' if text.find('아웃') > 0 else 'a'
    result = 'h' if text.find('홈런') > 0 else result
    
    beforeBase = 0
    afterBase = None
    if any([s in text for s in ['안타', '1루타', '4구', '볼넷', '출루',
                                '낫아웃 포일', '낫아웃 폭투', '낫아웃 다른주자',
                                '몸에 맞는', '실책', '타격방해', '야수선택']]):
        afterBase = 1
    elif '2루타' in text:
        afterBase = 2
    elif '3루타' in text:
        afterBase = 3
    elif '홈런' in text:
        afterBase = 4
    elif '아웃' in text:
        afterBase = None
    
    return [runner, result, beforeBase, afterBase]


def parse_runner_result(text):
    runner = text.split(' ')[1]
    
    result = 'o' if text.find('아웃') > 0 else 'a' # o : out, a : advance
    result = 'h' if text.find('홈인') > 0 else result # 'h' : home-in
    
    beforeBase = int(text[0])
    afterBase = None if result != 'a' else int(text[text.find('루까지')-1])
    afterBase = 4 if result == 'h' else afterBase
    
    return [runner, result, beforeBase, afterBase]


batter_results = [
    ['삼진', '삼진', '삼진'],
    ['볼넷', '볼넷', '볼넷'],
    ['자동 고의 4구', '고의 4구', '고의 4구'],
    ['자동 고의4구', '고의 4구', '고의 4구'],
    ['고의4구', '고의 4구', '고의 4구'],
    ['몸에', '몸에 맞는 공', '몸에 맞는 공'],
    ['1루타', '안타', '안타'],
    ['내야안타', '내야 안타', '내야 안타'],
    ['번트안타', '번트 안타', '번트 안타'],
    ['안타', '안타', '안타'],
    ['2루타', '2루타', '2루타'],
    ['3루타', '3루타', '3루타'],
    ['홈런', '홈런', '홈런'],
    ['낫아웃 폭투', '낫아웃 출루', '낫아웃 폭투'],
    ['낫아웃 포일', '낫아웃 출루', '낫아웃 포일'],
    ['낫 아웃', '삼진', '낫아웃 삼진'],
    ['낫아웃 다른주자 수비 실책', '낫아웃 출루', '낫아웃 다른 주자 수비 실책'],
    ['낫아웃 다른주자 수비', '낫아웃 출루', '낫아웃 다른 주자 포스 아웃'],
    ['땅볼로 출루', '포스 아웃', '땅볼 아웃'],
    ['땅볼 아웃', '포스 아웃', '땅볼 아웃'],
    ['플라이 아웃', '필드 아웃', '플라이 아웃'],
    ['인필드', '필드 아웃', '인필드 플라이'],
    ['파울플라이', '필드 아웃', '파울 플라이 아웃'],
    ['라인드라이브 아웃', '필드 아웃', '라인드라이브 아웃'],
    ['번트 아웃', '필드 아웃', '번트 아웃'],
    ['병살타', '병살타', '병살타'],
    ['희생번트 아웃', '희생 번트', '희생 번트'],
    ['희생플라이 아웃', '희생 플라이', '희생 플라이'],
    ['희생플라이아웃', '희생 플라이', '희생 플라이'],
    ['쓰리번트', '삼진', '쓰리번트 삼진'],
    ['타구맞음', '필드 아웃', '타구맞음 아웃'],
    ['희생번트 실책', '실책', '실책'],
    ['희생번트 야수선택', '야수 선택', '야수 선택'],
    ['야수선택', '야수 선택', '야수 선택'],
    ['실책', '실책', '실책'],
    ['타격방해', '타격 방해', '타격 방해'],
    ['삼중살', '삼중살', '삼중살'],
    ['부정타격', '필드 아웃', '부정 타격 아웃'],
    ['번트', '번트 안타', '안타'],
]


header_row = ['pitch_type', 'pitcher', 'batter', 'pitcher_ID', 'batter_ID',
              'speed', 'pitch_result', 'pa_result', 'pa_result_detail',
              'description', 'balls', 'strikes', 'outs',
              'inning', 'inning_topbot', 'score_away', 'score_home',
              'stands', 'throws', 'on_1b', 'on_2b', 'on_3b',
              'pos_1', 'pos_2', 'pos_3', 'pos_4', 'pos_5',
              'pos_6', 'pos_7', 'pos_8', 'pos_9',
              'px', 'pz', 'pfx_x', 'pfx_z', 'pfx_x_raw', 'pfx_z_raw',
              'x0', 'z0', 'sz_top', 'sz_bot',
              'y0', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az',
              'game_date', 'home', 'away', 'home_alias', 'away_alias',
              'stadium', 'referee', 'pa_number', 'pitch_number', 'pitchID']


def get_pitch_location_break(row):
    if not np.isnan(row[17]):
        return [None]*6
    else:
        ax = row[16]
        ay = row[18]
        az = row[19]
        vx0 = row[14]
        vy0 = row[12]
        vz0 = row[13]
        x0 = row[17]
        y0 = 50
        z0 = row[15]
        cpy = 1.4167

        # do math
        t = (-vy0 - (vy0 * vy0 - 2 * ay * (y0 - cpy)) ** 0.5) / ay

        t40 = (-vy0 - (vy0 * vy0 - 2 * ay * (y0 - 40)) ** 0.5) / ay
        x40 = x0 + vx0 * t40 + 0.5 * ax * t40 * t40
        vx40 = vx0 + ax * t40
        z40 = z0 + vz0 * t40 + 0.5 * az * t40 * t40
        vz40 = vz0 + az * t40
        th = t - t40
        x_no_air = x40 + vx40 * th
        z_no_air = z40 + vz40 * th - 0.5 * 32.174 * th * th
        z_no_induced = z0 + vz0 * t

        px = x0 + vx0 * t + ax * t * t * 0.5
        pz = z0 + vz0 * t + az * t * t * 0.5

        pfx_x = (px - x_no_air) * 12
        pfx_z = (pz - z_no_air) * 12
        pfx_x_raw = px * 12
        pfx_z_raw = (pz - z_no_induced) * 12

        return px, pz, pfx_x, pfx_z, pfx_x_raw, pfx_z_raw

In [383]:
def convert_row_to_save_format(row, fields, bases,
                                pa_result_details=None):
    # row: pandas Series
    save_row = {k: None for k in header_row}
    save_row['pitcher'] = pitcher_name
    save_row['batter'] = batter_name
    save_row['pitcher_ID'] = pitcher_code
    save_row['batter_ID'] = batter_code
    save_row['balls'] = balls
    save_row['strikes'] = strikes
    save_row['outs'] = outs
    save_row['inning'] = inn
    save_row['inning_topbot'] = '초' if top_bot == 0 else '말'
    save_row['score_away'] = score[0]
    save_row['score_home'] = score[1]
    
    if stands is not None:
        save_row['stands'] = stands[2]
    if throws is not None:
        save_row['throws'] = throws[0]
    
    save_row['pa_number'] = pa_number

    save_row['game_date'] = game_date
    save_row['home'] = home
    save_row['away'] = away
    save_row['home_alias'] = home_alias
    save_row['away_alias'] = away_alias
    save_row['stadium'] = stadium
    save_row['referee'] = referee
    
    for runner in bases:
        if runner[2] > 0:
            save_row[f'on_{runner[2]}b'] = runner[0]
    
    save_row['pos_1'] = fields['투수'].get('name')
    save_row['pos_2'] = fields['포수'].get('name')
    save_row['pos_3'] = fields['1루수'].get('name')
    save_row['pos_4'] = fields['2루수'].get('name')
    save_row['pos_5'] = fields['유격수'].get('name')
    save_row['pos_6'] = fields['3루수'].get('name')
    save_row['pos_7'] = fields['좌익수'].get('name')
    save_row['pos_8'] = fields['중견수'].get('name')
    save_row['pos_9'] = fields['우익수'].get('name')

    if row is not None:
        save_row['pitch_type'] = row[4]
        save_row['speed'] = row[6]
        save_row['pitch_result'] = row[2].split(' ')[-1]
        save_row['pitch_number'] = pitch_number
        save_row['pitchID'] = row[5]
        
        if len(row) > 13:
            if not np.isnan(row[17]):
                save_row['x0'] = row[17]
                save_row['z0'] = row[15]
                save_row['sz_top'] = row[10]
                save_row['sz_bot'] = row[11]
                px, pz, pfx_x, pfx_z, pfx_x_raw, pfx_z_raw = get_pitch_location_break(row)

                save_row['px'] = px
                save_row['pz'] = pz
                save_row['pfx_x'] = pfx_x
                save_row['pfx_z'] = pfx_z
                save_row['pfx_x_raw'] = pfx_x_raw
                save_row['pfx_z_raw'] = pfx_z_raw

                save_row['y0'] = 50
                save_row['vx0'] = row[14]
                save_row['vy0'] = row[12]
                save_row['vz0'] = row[13]
                save_row['ax'] = row[16]
                save_row['ay'] = row[18]
                save_row['az'] = row[19]

    if pa_result_details is not None:
        save_row['description'] = pa_result_details[0]
        save_row['pa_result'] = pa_result_details[1]
        save_row['pa_result_detail'] = pa_result_details[2]

    return save_row

In [933]:
game_id = '20161008HTHH02016'

pdf, bdf, rdf = get_game_data(game_id)

game_date = game_id[:8]
away = game_id[8:10]
home = game_id[10:12]
away_alias = pdf.loc[pdf.homeaway == 'a'].team_name.unique()[0]
home_alias = pdf.loc[pdf.homeaway == 'h'].team_name.unique()[0]
stadium = rdf.stadium.unique()[0]
referee = rdf.referee.unique()[0]

########################
# 라인업 & 필드 채우기 #
########################

lineups = [[], []] # 초 공격 / 말 공격
fields = [{}, {}] # 초 수비 / 말 수비

bats = [bdf.loc[bdf.homeaway == 'a'], bdf.loc[bdf.homeaway == 'h']]

pits = [pdf.loc[pdf.homeaway == 'a'], pdf.loc[pdf.homeaway == 'h']]

abat_seqno_min = bats[0].loc[bats[0].batOrder.between(1, 9)].groupby('batOrder').seqno.min().tolist()
hbat_seqno_min = bats[1].loc[bats[1].batOrder.between(1, 9)].groupby('batOrder').seqno.min().tolist()

for i in range(9):
    aCode = bats[0].loc[(bats[0].batOrder == i+1) & (bats[0].seqno == abat_seqno_min[i])].pCode.values[0]
    aName = bats[0].loc[(bats[0].batOrder == i+1) & (bats[0].seqno == abat_seqno_min[i])].name.values[0]
    hCode = bats[1].loc[(bats[1].batOrder == i+1) & (bats[1].seqno == hbat_seqno_min[i])].pCode.values[0]
    hName = bats[1].loc[(bats[1].batOrder == i+1) & (bats[1].seqno == hbat_seqno_min[i])].name.values[0]
    aHitType = bats[0].loc[bats[0].pCode == aCode].hitType.values[0]
    hHitType = bats[1].loc[bats[1].pCode == hCode].hitType.values[0]
    aPlayer = {'name': aName, 'code': aCode, 'hitType': aHitType}
    hPlayer = {'name': hName, 'code': hCode, 'hitType': hHitType}
    
    aPos = bats[0].loc[bats[0].pCode == aCode].posName.values[0]
    hPos = bats[1].loc[bats[1].pCode == hCode].posName.values[0]
    fields[1][aPos] = aPlayer
    fields[0][hPos] = hPlayer
    
    aLineup = {'name': aName, 'code': aCode, 'pos': aPos, 'hitType': aHitType}
    hLineup = {'name': hName, 'code': hCode, 'pos': hPos, 'hitType': hHitType}
    lineups[0].append(aLineup)
    lineups[1].append(hLineup)

aPitcher = {'name': pits[0].iloc[0]['name'], 'code': pits[0].iloc[0].pCode, 'hitType': pits[0].iloc[0].hitType}
hPitcher = {'name': pits[1].iloc[0]['name'], 'code': pits[1].iloc[0].pCode, 'hitType': pits[0].iloc[0].hitType}
fields[1]['투수'] = aPitcher
fields[0]['투수'] = hPitcher

home_bdf = bdf.loc[bdf.homeaway == 'h']
away_bdf = bdf.loc[bdf.homeaway == 'a']
home_pdf = pdf.loc[pdf.homeaway == 'h']
away_pdf = pdf.loc[pdf.homeaway == 'a']

batter_list_cols = ['name', 'pCode', 'posName', 'hitType', 'batOrder', 'seqno']
pitcher_list_cols = ['name', 'pCode', 'hitType', 'seqno']
home_batter_list = home_bdf[batter_list_cols].values.tolist()
home_pitcher_list = home_pdf[pitcher_list_cols].values.tolist()
away_batter_list = away_bdf[batter_list_cols].values.tolist()
away_pitcher_list = away_pdf[pitcher_list_cols].values.tolist()

rdf_cols = ['textOrder', 'seqno', 'text', 'type',
            'stuff', 'pitchId', 'speed', 'referee', 'stadium']
if 'x0' in rdf.columns:
    rdf_cols += ['crossPlateX', 'topSz', 'bottomSz',
                 'vy0', 'vz0', 'vx0', 'z0', 'ax', 'x0', 'ay', 'az']
    
for col in ['outPlayer', 'inPlayer', 'shiftPlayer']:
    if col in rdf.columns:
        rdf_cols.append(col)

rdf_array = rdf[rdf_cols].sort_values(['textOrder', 'seqno']).values

In [934]:
debug_mode = True

runner_bases = []

start_time = time.time()

score = [0, 0] # 초공(어웨이), 말공(홈)
balls, strikes, outs = 0, 0, 0
batter_name, batter_code, _pos, stands = lineups[0][0].values()
pitcher_name, pitcher_code, throws = fields[1].get('투수').values()
DH_exist = [True, True]
DH_exist_after = [True, True]
last_pitch = None

top_bot = 1 # 0 : 초, 1 : 말
# fields[1] : 말 수비, fields[0] : 초 수비
# lineups[1] : 말 공격 타순, lineups[0] : 초 공격 타순
cur_order = 1

pa_number = 0
pitch_number = 0

print_rows = []

pitch_result = None
pa_result = None
pa_result_detail = None
description = ''
last_pitch = None

try:
    ind = 0
    inn = 0
    top_bot = 1
    while ind < rdf_array.shape[0]:        
        row = rdf_array[ind]
        cur_type = row[3]
        cur_text = row[2]
        cur_to = row[0]
        ##############################
        ###### type에 따라 파싱 ######
        ##############################

        if cur_type == 1:
            last_pitch = row
            pitch_number += 1
            res = cur_text.split(' ')[-1]
            pitch_result = res
            # 인플레이/삼진/볼넷 이외에는 여기서 row print
            # 몸에맞는공은 타석 결과에서 수정
            if res != '타격':
                save_row = convert_row_to_save_format(row, fields[top_bot], runner_bases)
                print_rows.append(save_row)

            if res == '볼':
                balls += 1
            elif res == '스트라이크':
                strikes += 1
            elif res.find('헛스윙') > -1:
                strikes += 1
            elif res.find('파울') > -1:
                strikes = strikes+1 if strikes < 2 else 2
            if (strikes > 3) or (balls > 4) or (outs > 3):
                if debug_mode is True:
                    print('3S/4B/3O')
                assert False
            ind = ind + 1

        elif cur_type == 8:
            # 타석 시작(x번타자 / 대타)
            position = cur_text.split(' ')[0]
            last_pitch = None
            pa_result = None
            pitch_result = None
            pa_result_detail = None
            description = ''
            if len(position) > 2:
                cur_order = int(position[0])
                batter_name, batter_code, _pos, stands = lineups[top_bot][cur_order - 1].values()
                pitch_number = 0
                pa_number += 1
                balls, strikes = 0, 0

                # 버그 : 대타 교체 텍스트가 누락된 경우. 20170902HTWO02017
                # 임시조치에 불과함~
                if batter_name != cur_text.split(' ')[1]:
                    print('또 버그')
                    print(cur_text, batter_name)
                    batter_name = cur_text.split(' ')[1]
                    lineups[top_bot][cur_order - 1]['name'] = batter_name
                    if top_bot == 0:
                        for p in away_batter_list:
                            if (p[0] == batter_name) & (p[4] == cur_order):
                                batter_code = p[1]
                                stands = p[3]
                                break
                    else:
                        for p in home_batter_list:
                            if (p[0] == batter_name) & (p[4] == cur_order):
                                batter_code = p[1]
                                stands = p[3]
                                break
                    lineups[top_bot][cur_order - 1]['code'] = batter_code
                    lineups[top_bot][cur_order - 1]['hitType'] = stands

                runner_bases.append([batter_name, batter_code, 0, [5]])
            else:
                batter_name, batter_code, _pos, stands = lineups[top_bot][cur_order - 1].values()
                if batter_name != cur_text.split(' ')[1]:
                    print('또 버그')
                    print(cur_text, batter_name)
                    if len(cur_text.strip().split(' ')) > 1:
                        batter_name = cur_text.split(' ')[1]
                        lineups[top_bot][cur_order - 1]['name'] = batter_name
                        if top_bot == 0:
                            for p in away_batter_list:
                                if (p[0] == batter_name) & (p[4] == cur_order):
                                    batter_code = p[1]
                                    stands = p[3]
                                    break
                        else:
                            for p in home_batter_list:
                                if (p[0] == batter_name) & (p[4] == cur_order):
                                    batter_code = p[1]
                                    stands = p[3]
                                    break
                        lineups[top_bot][cur_order - 1]['code'] = batter_code
                        lineups[top_bot][cur_order - 1]['hitType'] = stands

                        for runner in runner_bases:
                            if (runner[2] == 0) & (runner[0] != batter_name):
                                runner[0] = batter_name
                                runner[1] = batter_code
                                break
            ind = ind + 1
        elif (cur_type == 13) or (cur_type == 23):
            # 타자주자(비득점/득점)
            runner_stack = []
            cur_ind = ind
            cur_row = rdf_array[cur_ind]
            description = ''

            while ((cur_type == 13) or\
                   (cur_type == 23) or\
                   (cur_type == 14) or\
                   (cur_type == 24)):
                runner_stack.append(cur_row)
                description += cur_row[2].strip() + '; '
                cur_ind += 1

                if rdf_array[cur_ind][0] != cur_to:
                    break
                cur_row = rdf_array[cur_ind]
                cur_type = cur_row[3]
            description = description.strip()

            result = parse_batter_result(cur_text)
            pa_result = result[0]
            pa_result_detail = result[1]

            if pitch_result != '타격':
                print_rows[-1]['description'] = description
                print_rows[-1]['pa_result'] = pa_result
                print_rows[-1]['pa_result_detail'] = pa_result_detail
            else:
                save_row = convert_row_to_save_format(last_pitch, fields[top_bot], runner_bases,
                                                      [description, pa_result, pa_result_detail])
                print_rows.append(save_row)

            ind = cur_ind
            
            if len(runner_bases) == 0:
                if debug_mode is True:
                    print('base error')
                assert False
            cur_runner = runner_bases[0]
            cur_runner_ind = 0
            for rrow in runner_stack[::-1]:
                if (rrow[3] == 13) or (rrow[3] == 23):
                    runner_name, run_result,\
                    runner_before_base, runner_after_base = parse_batter_as_runner(rrow[2])
                else:
                    runner_name, run_result,\
                    runner_before_base, runner_after_base = parse_runner_result(rrow[2])

                base_loop_num = 0
                while not ((cur_runner[0] == runner_name) &\
                           (cur_runner[2] <= runner_before_base) &\
                           (cur_runner[3][-1] >= runner_before_base)):
                    cur_runner_ind = cur_runner_ind + 1
                    if cur_runner_ind >= len(runner_bases):
                        cur_runner_ind = 0
                    cur_runner = runner_bases[cur_runner_ind]
                    base_loop_num += 1
                    if base_loop_num > 4:
                        if debug_mode is True:
                            print('base error')
                        assert False

                if cur_runner[3][-1] == 5:
                    if runner_after_base is not None:
                        cur_runner[3] = [runner_after_base, runner_before_base]
                    elif runner_before_base != 0:
                        cur_runner[3] = [runner_after_base, runner_before_base]
                else:
                    cur_runner[3].append(runner_before_base)

                if run_result == 'h':
                    score[top_bot] += 1
                elif run_result == 'o':
                    outs += 1

            after_runner_bases = []

            for runner in runner_bases:
                # name, code, src, route
                if (runner[2] == 0) & (runner[3][0] == 5):
                    continue
                elif runner[3][0] == 5:
                    after_runner_bases.append(runner[:])
                elif (runner[3][0] is not None) & (runner[3][0] != 4):
                    after_runner_bases.append([runner[0], runner[1], runner[3][0], [5]])
            runner_bases = after_runner_bases[:]
        elif (cur_type == 14) or (cur_type == 24):
            # 주자(비득점/득점)
            runner_stack = []
            cur_ind = ind
            cur_row = rdf_array[cur_ind]
            description = ''
            while (cur_type == 14) or (cur_type == 24):
                runner_stack.append(cur_row)
                description += cur_row[2] + '; '
                cur_ind += 1
                if rdf_array[cur_ind][0] != cur_to:
                    break
                cur_row = rdf_array[cur_ind]
                cur_type = cur_row[3]
            description = description.strip()

            save_row = convert_row_to_save_format(None, fields[top_bot], runner_bases,
                                                  [description, None, None])
            print_rows.append(save_row)
            ind = cur_ind
            
            if len(runner_bases) == 0:
                if debug_mode is True:
                    print('base error')
                assert False
            cur_runner = runner_bases[0]
            cur_runner_ind = 0

            for rrow in runner_stack[::-1]:
                runner_name, run_result,\
                runner_before_base, runner_after_base = parse_runner_result(rrow[2])

                base_loop_num = 0
                while not ((cur_runner[0] == runner_name) &\
                            (cur_runner[2] <= runner_before_base) &\
                            (cur_runner[3][-1] >= runner_before_base)):
                    cur_runner_ind = cur_runner_ind + 1
                    if cur_runner_ind >= len(runner_bases):
                        cur_runner_ind = 0
                    cur_runner = runner_bases[cur_runner_ind]
                    base_loop_num += 1
                    if base_loop_num > 4:
                        if debug_mode is True:
                            print('base error')
                        assert False

                if cur_runner[3][-1] == 5:
                    cur_runner[3] = [runner_after_base, runner_before_base]
                else:
                    cur_runner[3].append(runner_before_base)

                if run_result == 'h':
                    score[top_bot] += 1
                elif run_result == 'o':
                    outs += 1

            after_runner_bases = []

            for runner in runner_bases:
                # name, code, src, route
                if runner[3][0] == 5:
                    after_runner_bases.append(runner[:])
                else:
                    if (runner[3][0] is not None) & (runner[3][0] != 4):
                        after_runner_bases.append([runner[0], runner[1], runner[3][0], [5]])
            runner_bases = after_runner_bases[:]
        elif cur_type == 0:
            # 이닝 시작
            if top_bot == 1:
                inn += 1
            top_bot = (1 - top_bot)
            pitcher_name, pitcher_code, throws = fields[top_bot].get('투수').values()

            runner_bases = []
            balls, outs, strikes = 0, 0, 0
            DH_exist_after = DH_exist[:]
            last_pitch = None

            pitch_number = 0
            ind = ind + 1
        elif cur_type == 2:
             # 교체/변경
            text_stack = []
            cur_ind = ind
            cur_row = rdf_array[cur_ind]
            description = ''

            while cur_row[3] == 2:
                text_stack.append(cur_row[2])
                description += cur_row[2] + '; '
                cur_ind = cur_ind +1
                if rdf_array[cur_ind][0] != cur_to:
                    break
                cur_row = rdf_array[cur_ind]
            description = description.strip()
            ind = cur_ind

            save_row = convert_row_to_save_format(None, fields[top_bot], runner_bases,
                                                  [description, None, None])
            print_rows.append(save_row)

            change_stack = []
            for text in text_stack:
                order = None
                before_pos = text.split(' ')[0]

                if text.find('변경') > 0:
                    after_pos = text.split('(')[0].strip().split(' ')[-1]
                    shift_name = text.split(' ')[1].strip()
                    shift_code = None
                    shift_hittype = None

                    for j in range(9):
                        if (lineups[1 - top_bot][j].get('name') == shift_name) &\
                           (lineups[1 - top_bot][j].get('pos') == before_pos):
                            shift_code = lineups[1 - top_bot][j].get('code')
                            shift_hittype = lineups[1 - top_bot][j].get('hitType')
                            order = j + 1
                            break
                    if before_pos == '지명타자':
                        DH_exist_after[top_bot] = False
                    if after_pos == '투수':
                        DH_exist_after[top_bot] = False
                    ########################################################################
                    ##### 대타 출장 후 같은 이닝에 타순 1바퀴 돌면서 포지션 변경하는 경우
                    ########################################################################
                    if (before_pos == '대타') & (order == None):
                        for i in range(9):
                            if (lineups[top_bot][j].get('name') == shift_name) &\
                               (lineups[top_bot][j].get('pos') == before_pos):
                                lineups[top_bot][j]['pos'] = after_pos
                                break
                        continue

                    change = [before_pos, after_pos, order, shift_name, shift_code, shift_hittype]
                    change_stack.append(change)
                else:
                    after_pos = text.split('(')[0].strip().split(' ')[3]
                    after_name = text.split('(')[0].strip().split(' ')[-1]
                    before_name = text.split(' ')[1].strip()
                    before_code = None
                    after_code = None
                    after_hittype = None
                    order = None

                    if (before_pos == '지명타자') & (after_pos !='지명타자') & (after_pos !='대타'):
                        DH_exist_after[top_bot] = False
                    elif (before_pos == '투수') & (after_pos != '투수'):
                        DH_exist_after[top_bot] = False
                    elif (before_pos != '투수') & (after_pos == '투수'):
                        DH_exist_after[top_bot] = False

                    if before_pos.find('번타자') > 0:
                        order = int(before_pos[0])
                        before_code = lineups[top_bot][order - 1].get('code')
                        
                        if top_bot == 0:
                            for j in range(len(away_batter_list)):
                                if away_batter_list[j][1] == before_code:
                                    after_code = away_batter_list[j+1][1]
                                    after_hittype = away_batter_list[j+1][3]
                                    break
                        else:
                            for j in range(len(home_batter_list)):
                                if home_batter_list[j][1] == before_code:
                                    after_code = home_batter_list[j+1][1]
                                    after_hittype = home_batter_list[j+1][3]
                                    break
                    elif after_pos == '투수':
                        # 예외처리
                        if (after_name == pitcher_name) & (before_pos != '투수'):
                            after_code = pitcher_code
                            after_hittype = throws
                        elif (after_name != pitcher_name) & (before_pos != '투수'):
                            if top_bot == 0:
                                for j in range(len(home_pitcher_list)):
                                    if home_pitcher_list[j][1] == pitcher_code:
                                        after_code = home_pitcher_list[j+1][1]
                                        after_hittype = home_pitcher_list[j+1][2]
                                        break
                            else:
                                for j in range(len(away_pitcher_list)):
                                    if away_pitcher_list[j][1] == pitcher_code:
                                        after_code = away_pitcher_list[j+1][1]
                                        after_hittype = away_pitcher_list[j+1][2]
                                        break
                        else:
                            before_code = fields[top_bot][before_pos].get('code')
                            if top_bot == 0:
                                for j in range(len(home_pitcher_list)):
                                    if home_pitcher_list[j][1] == before_code:
                                        after_code = home_pitcher_list[j+1][1]
                                        after_hittype = home_pitcher_list[j+1][2]
                                        break
                            else:
                                for j in range(len(away_pitcher_list)):
                                    if away_pitcher_list[j][1] == before_code:
                                        after_code = away_pitcher_list[j+1][1]
                                        after_hittype = away_pitcher_list[j+1][2]
                                        break
                        if ((DH_exist[top_bot] is False) or\
                            (DH_exist_after[top_bot] != DH_exist[top_bot])):
                            for j in range(9):
                                if (lineups[1 - top_bot][j].get('name') == before_name) &\
                                   (lineups[1 - top_bot][j].get('pos') == before_pos):
                                    order = i + 1
                                    break
                    elif before_pos.find('루주자') > 0:
                        before_base = int(before_pos[0])
                        for runner in runner_bases:
                            if (runner[0] == before_name) & (runner[2] == before_base):
                                before_code = runner[1]
                                break
                        for j in range(9):
                            if lineups[top_bot][j].get('code') == before_code:
                                order = j + 1
                                break
                        if top_bot == 0:
                            for j in range(len(away_batter_list)):
                                if away_batter_list[j][1] == before_code:
                                    after_code = away_batter_list[j+1][1]
                                    after_hittype = away_batter_list[j+1][3]
                                    break
                        else:
                            for j in range(len(home_batter_list)):
                                if home_batter_list[j][1] == before_code:
                                    after_code = home_batter_list[j+1][1]
                                    after_hittype = home_batter_list[j+1][3]
                                    break
                    else:
                        for j in range(9):
                            if (lineups[1 - top_bot][j].get('name') == before_name) &\
                               (lineups[1 - top_bot][j].get('pos') == before_pos):
                                before_code = lineups[1 - top_bot][j].get('code')
                                order = j + 1
                                break
                        if order is None:
                            for j in range(9):
                                if (lineups[top_bot][j].get('name') == before_name) &\
                                   (lineups[top_bot][j].get('pos') == before_pos):
                                    before_code = lineups[top_bot][j].get('code')
                                    order = j + 1
                                    break
                        if order is None:
                            if debug_mode is True:
                                print('cant find player name/position in lineup')
                            assert False
                        if top_bot == 0:
                            for j in range(len(home_batter_list)):
                                if home_batter_list[j][1] == before_code:
                                    after_code = home_batter_list[j+1][1]
                                    after_hittype = home_batter_list[j+1][3]
                                    break
                            if after_code is None:
                                for j in range(len(away_batter_list)):
                                    if away_batter_list[j][1] == before_code:
                                        after_code = away_batter_list[j+1][1]
                                        after_hittype = away_batter_list[j+1][3]
                                        break
                        else:
                            for j in range(len(away_batter_list)):
                                if away_batter_list[j][1] == before_code:
                                    after_code = away_batter_list[j+1][1]
                                    after_hittype = away_batter_list[j+1][3]
                                    break
                            if after_code is None:
                                for j in range(len(home_batter_list)):
                                    if home_batter_list[j][1] == before_code:
                                        after_code = home_batter_list[j+1][1]
                                        after_hittype = home_batter_list[j+1][3]
                                        break
                        if after_code is None:
                            if debug_mode is True:
                                print('cant find player code in lineup')
                            assert False
                        for j in range(len(runner_bases)):
                            if runner_bases[j][1] == before_code:
                                runner_bases[j][0] = after_name
                                runner_bases[j][1] = after_code
                                break
                    change = [before_pos, after_pos, order, after_name, after_code, after_hittype]
                    change_stack.append(change)

            for i in range(len(change_stack)):
                change = change_stack[i]
                before_pos, after_pos, order, after_name, after_code, after_hittype = change
                    
                if (after_pos != '대타') & (after_pos != '대주자'):
                    fields[top_bot][after_pos]['code'] = after_code
                    fields[top_bot][after_pos]['name'] = after_name
                    fields[top_bot][after_pos]['hitType'] = after_hittype

                if order is not None:
                    if (after_pos != '대타') & (after_pos != '대주자'):
                        tb = 1 - top_bot
                    else:
                        tb = top_bot

                    lineups[tb][order - 1]['code'] = after_code
                    lineups[tb][order - 1]['name'] = after_name
                    lineups[tb][order - 1]['pos'] = after_pos
                    lineups[tb][order - 1]['hitType'] = after_hittype

                if after_pos == '투수':
                    pitcher_code = after_code
                    pitcher_name = after_name
                    throws = after_hittype
                elif after_pos == '대주자':
                    before_base = int(before_pos[0])
                    for j in range(len(runner_bases)):
                        if runner_bases[j][2] == before_base:
                            runner_bases[j][0] = after_name
                            runner_bases[j][1] = after_code
                            break
                elif after_pos == '대타':
                    runner_bases[-1][0] = after_name
                    runner_bases[-1][1] = after_code
                    stands = after_hittype
            DH_exist[top_bot] = DH_exist_after[top_bot]
        elif cur_type == 7:
            ind = ind + 1
        else:
            ind = ind + 1
except:
    ind = len(print_rows)-1
    while ind >= 0:
        if print_rows[ind]['pa_number'] == pa_number:
            print_rows = print_rows[:-1]
            ind -= 1
        else:
            break
    
    print("-"*60)
    print(f"=== gameid : {game_id}")
    traceback.print_exc(file=sys.stdout)
    print("-"*60)

elapsed_time = time.time() - start_time
print(f'{elapsed_time:.2f} sec')

또 버그
8번타자 이홍구 나지완
또 버그
1번타자 김호령 이홍구
base error
------------------------------------------------------------
=== gameid : 20161008HTHH02016
Traceback (most recent call last):
  File "<ipython-input-934-6444a309182d>", line 175, in <module>
    assert False
AssertionError
------------------------------------------------------------
0.01 sec


In [935]:
rdf.loc[rdf.textOrder.between(cur_to-2, cur_to)].loc[:, :'type']

,textOrder,seqno,text,type
673,112,637,9번타자 윤정우,8
675,112,638,1구 번트파울,1
676,112,639,2구 헛스윙,1
677,112,640,3구 타격,1
674,112,641,윤정우 : 유격수 땅볼 아웃 (유격수->1루수 송구아웃),13
668,113,642,1번타자 김호령,8
669,113,643,1구 헛스윙,1
670,113,644,2구 볼,1
671,113,645,3구 파울,1
672,113,646,4구 볼,1


In [936]:
print([x[2] for x in runner_stack])
print(rrow[2])
print(runner_bases)

['김호령 :  삼진 아웃 ']
윤정우 : 유격수 땅볼 아웃 (유격수->1루수 송구아웃)
[]


In [937]:
prdf = pd.DataFrame(print_rows)
prdf.tail(10).loc[:, 'batter':'on_3b']

,batter,pitcher_ID,batter_ID,speed,pitch_result,pa_result,pa_result_detail,description,balls,strikes,outs,inning,inning_topbot,score_away,score_home,stands,throws,on_1b,on_2b,on_3b
391,이성열,60636,73136,144,타격,실책,실책,이성열 : 유격수 실책으로 출루 (유격수 실책->1루수); 3루주자 김태균 : 홈인;,0,2,2,9,말,5,4,좌,좌,None,None,김태균
392,장민석,60636,71347,144,타격,필드 아웃,플라이 아웃,장민석 : 2루수 플라이 아웃;,0,0,2,9,말,5,5,좌,좌,이성열,None,None
393,이홍구,74857,63634,119,볼,None,None,None,0,0,0,10,초,5,5,우,좌,None,None,None
394,이홍구,74857,63634,125,스트라이크,None,None,None,1,0,0,10,초,5,5,우,좌,None,None,None
395,이홍구,74857,63634,137,스트라이크,None,None,None,1,1,0,10,초,5,5,우,좌,None,None,None
396,이홍구,74857,63634,123,볼,None,None,None,1,2,0,10,초,5,5,우,좌,None,None,None
397,이홍구,74857,63634,124,헛스윙,삼진,삼진,이홍구 : 삼진 아웃;,2,2,0,10,초,5,5,우,좌,None,None,None
398,윤정우,74857,61652,127,번트파울,None,None,None,0,0,1,10,초,5,5,우,좌,None,None,None
399,윤정우,74857,61652,121,헛스윙,None,None,None,0,1,1,10,초,5,5,우,좌,None,None,None
400,윤정우,74857,61652,123,타격,포스 아웃,땅볼 아웃,윤정우 : 유격수 땅볼 아웃 (유격수->1루수 송구아웃);,0,2,1,10,초,5,5,우,좌,None,None,None


# 버그 / 문제 있는 경기

모바일에서 PC와 다른 중계 텍스트가 나와서 문제.

텍스트 생략됐거나 순서가 바뀌었거나 아예 텍스트가 다른 경우가 대부분

라인업 데이터에 우투좌타 같은게 없는 경우도 있음(이건 생략)

X번타자 A : 대타 B로 교체 -> 텍스트가 '중견수 A : 중견수 B로 교체' 이런 경우도 있음.

따로 해결책은 없고 경기 별로 수술하거나 파싱 도중에 멈추고 거기까지만 결과를 출력

## 2016
- 20160709SSHH02016
    - 중계 기록 누락
- 20160712HHLG02016
    - 중계 누락
- 20160712LTSS02016
    - 중계 누락
- 20160712OBNC02016
    - 중계 누락
- 20160712SKHT02016
    - 중계 누락
- 20160712WOKT02016
    - 중계 누락
- 20160713SKHT02016
    - 중계 누락
- 20160713WOKT02016
    - 중계 오류
- 20160714SKHT02016
    - 중계 오류
- 20160723SSKT02016
    - 중계 오류(스트 중복)
- 20160724NCHT02016
    - 중계 오류(볼/스트 중복)
- 20160726SKHH02016
    - 중계 오류
- 20160727NCSS02016
    - 중계 누락
- 20160729WOSS02016
    - 중계 오류 (0루 주자)
- 20160807OBLT02016
    - 중계 오류
- 20160819HHLG02016
    - 중계 오류 (죽은사람 주자로 출루)
- 20160824HTNC02016
    - 중계 오류(볼/스트 중복)
- 20160903HHWO02016
    - 중계 오류 (죽은 주자 살아남)
- 20160907KTSS02016
    - 중계 오류(볼/스트 중복)
- 20160916LTHH02016
    - 중계 오류(3스트 삼진x)
- 20161004WONC02016
    - 중계 오류
- 20161007KTNC02016
    - 중계 오류(볼/스트 중복)
- 20161008HTHH02016
    - 중계 누락

## 2017
- 20170401HTSS02017
    - 중계기록 꼬임
- 20170407LGLT02017
    - 중계기록 꼬임. 1루주자->대주자 교체를 1루수->1루수로 표기
- 20170414OBNC02017
    - 중계기록 꼬임. 8회말 이상호가 두번 홈인
- 20170426OBWO02017
    - 중계기록 꼬임. 양의지 1루타가 2루타로 둔갑
- 20170512WOSS02017
    - 중계기록 꼬임
- 20170521LTLG02017
    - 중계기록 꼬임
- 20170526SSWO02017
    - 마지막에 파일이 끊겨있음
- 20170609KTNC02017
    - '좌익수 이대형' 교체가 '중견수 이대형' 교체로 되어있음, 로직 에러
- 20170801KTHT02017
    - 5회초 텍스트 꼬임, 1번 이대형 타석 누락
- 20170803OBSS02017
    - 6회말 대타 타석 텍스트 꼬임
- 20170815OBLT02017
    - 5회초 시작 동시에 텍스트 꼬임
- 20170825HTHH02017
    - 5회말 마지막타석 헛스윙 중복(4스트)
- 20170829HTSS02017
    - 4회말 이원석 2루타 -> 1루타로 둔갑
- 20170910WOSK02017
    - 중계 꼬임
- 20170915SKOB02017
    - 중계 꼬임
- 20171001SSLG02017
    - 중계 누락

## 2018
- 20180426HHHT02018
    - 중계 꼬임
- 20181013WOSS02018
    - 라인업 데이터 누락으로 로직 에러(주효상)

## 2019
- 20190424LTHH02019
    - 라인업 데이터 누락으로 로직 에러(오현택)
- 20190804KTWO02019
    - 볼 2번(5볼), 스트라이크 2번 (4스트)
- 20190816SKHT02019
    - 중계 꼬임
- 20190929SSKT02019
    - 중계 꼬임, 라인업 데이터 꼬임(0번 타자)

# 선수 교체

2인 이상 동시 포지션 변경 혹은 교체시 라인업/수비오더 처리 문제.

순서대로 처리하는 과정에서 out/in 코드 처리가 꼬일 가능성 있음.

ex)
1루수 A : 2루수 D(으)로 교체
2루수 B : 3루수 E(으)로 교체
3루수 C : 1루수 F(으)로 교체

수비 포지션뿐 아니라 타선까지 교체하는데 이 과정에서 타순은 다음 순서로 바뀐다.

1. '1루수' A의 타순에 D를 넣는다.
2. 2루수 자리에 D를 넣는다.
3. '2루수' D의 타순을 가져온다.
4. 원래 B 타순에 E를 넣어야 하지만 A 타순에 들어간다.
5. ...

이렇게 해서 망한다.

### 해결책 : 이번에도 스택 쌓아서 before/after 구분한다.

위 ex를 가져오면

before | after
- | -
1루수 A | 2루수 D
2루수 B | 3루수 E
3루수 C | 1루수 F

스택 쌓아서 순서대로 읽고 기록하면 이렇게 된다.

before POS | before ORDER | after POS | after ORDER
- | - | - | -
1루수 A | 1번 A | 2루수 D | 1번 D
2루수 B | 2번 B | 3루수 E | 2번 E
3루수 C | 3번 C | 1루수 F | 3번 F

이러면 after 값만 보고 쓱쓱 바꾸면 된다.

경우의 수

before POS | before ORDER | after POS | after ORDER | 교체/변경
- | - | - | - | -
__투수__ | X | __투수__ | X | __교체__
__투수__ | O | 야수 | O | __교체__
__투수__ | X | 야수 | O | __교체__
__투수__ | O | 야수 | O | 변경
야수/지명타자 | O | 야수 | O | __교체__
야수/지명타자 | O | 야수 | O | 변경
야수/지명타자 | O | __투수__ | O | __교체__
야수/지명타자 | O | __투수__ | O | 변경
1번타자 | O | 대타 | O | __교체__
1루주자 | O | 대주자 | O | __교체__
대타 | O | 야수 | O | __교체__
대타 | O | 야수 | O | 변경
대주자 | O | 야수 | O | __교체__
대주자 | O | 야수 | O | 변경

# Structure

Top-Bottom 식으로 밑그림을 그리고 가보자.

## Read Files

경기 별로 파일은 3가지가 저장된다.
1. pbp_data/{Year}/{Month}/{gameID}_relay.csv
2. pbp_data/{Year}/{Month}/{gameID}_batting.csv
3. pbp_data/{Year}/{Month}/{gameID}_pitching.csv



# 1. `relay` 중계 파일 구조

`csv` 형식으로 저장했는데 각 컬럼의 의미는 이렇다.

columns| definition
-|-
textOrder| 텍스트 순번(타석, 메시지 단위로 구분)
seqno| 같은 `textOrder` 안에서 출력 순서
text | 중계 텍스트 내용
type | 텍스트 타입
stuff | 구종
pitchNum | 타석 투구수(신뢰 x)
pitchResult | 스트(T)/볼(B)/파울(F)/헛스윙(S)
pitchId | 트래킹데이터 고유값 ID
speed | 문자중계 찍히는 구속
referee | 구심
stadium | 구장
outPlayer | 교체시/나가는 선수
inPlayer | 교체시/들어온 선수
shiftPlayer | 포변/포변 선수
inn | 이닝(신뢰 x)
ballcount | 타석 투구수(신뢰 x)
기타 | vx0, vy0, ... 트래킹 데이터

## type 의미

- `0` : x회초(말) xx 공격
- `1` : 투구 (x구 ...)
- `2` : 교체
- `7` : 시스템 메시지 (비디오 판독, ...)
- `8` : x번타자 xxx
- `13` : 타자주자 타석 결과(득점 없는 타석)
- `14` : 주자 주루/아웃(득점 없는 타석)
- `23` : 타자주자 타석 결과(득점 있는 타석)
- `24` : 주자 주루/아웃(득점 있는 타석)
- `44` : 파울 에러 (pass)
- `99` : 경기 종료 메시지

## Row print하는 경우

- 1 : 투구 직후
- 2 : 교체 이후
- 13, 23 : 타석 결과
- 14, 24 : 주루 결과

# 2. `batting` 파일 구조 / 야수 관련 `object` 설정

`csv` 형식이며 각 컬럼 의미는 이렇다.

columns | definition
-|-
name | 선수 이름
pCode | 선수 코드
pos | 포지션(한글) - 선발선수는 선발시 포지션, 나머지는 경기종료시 포지션
hitType | 투타
seqno | 해당 타순 선수들의 출전 순서(낮으면 선발)
batOrder | 타순
homeaway | `h`는 홈, `a`는 어웨이
ab, hit, ... | 경기 기록

## 초기 `object` 설정

`csv` 파일을 `pd.DataFrame` 형식으로 로드해둔다.

로드 이후 다음 순서로 경기 시작시 배팅 오더/수비 라인업 오브젝트를 구성한다.

1. `batOrder` 순으로 순회한다.
2. 각 `batOrder`에서 `seqno` 값이 가장 낮은 row를 뽑아 라인업 오더에 넣는다.
    - 2-1. `name`, `pCode`만 넣어둔다. 세부 정보 필요시 `pCode`로 `DataFrame`을 조회한다.
    - 2-2. 해당 row의 포지션으로 선발 수비 라인업도 구성한다.
3. 수비라인업의 투수 설정을 한다. (#3. `pitching` 파일 구조 참조)

## 조회하는 경우
경기 중에 배팅 오더/수비 라인업 오브젝트를 조회하는 경우는 다음이 있다.

1. 선수 교체시
    - 1-1. 대타 : 배팅 오더 교체, 나간선수 수비 라인업 `nullify`
    - 1-2. 투교 : 수비 라인업 `refresh`
    - 1-3. 포변 : 수비 라인업 `refresh`
2. 1구 단위 row 기록할 때 : 수비 라인업 조회하고 기록
3. 타석 바뀔 때: 타순 보고 선수 대조

# 3. `pitching ` 파일 구조 / 투수 관련 `object` 설정

`csv` 형식이며 각 컬럼 의미는 이렇다.

columns | definition
-|-
name | 선수 이름
pCode | 선수 코드
hitType | 투타
seqno | 출전 순서(낮으면 선발)
homeaway | `h`는 홈, `a`는 어웨이
inn, run, er, ... | 경기 기록


## 초기 `object` 설정

`csv` 파일을 `pd.DataFrmae` 형식으로 로드해둔다.

로드 이후 다음 순서로 경기 시작시 배팅 오더/수비 라인업 오브젝트를 구성한다.

1. 야수쪽 수비 라인업 설정을 마친뒤 투수도 수비 라인업에 설정한다.
  1-1. 야수와 마찬가지로 `name`, `pCode`만 넣어둔다.
  1-2. `seqno` 가장 낮은 투수를 집어넣는다.

## 조회하는 경우

경기 중에 투수 라인업을 조회하는 경우는 다음이 있다.

1. 선수 교체시
  1-1. 대타: 아주 가끔 투수가 대타로 나갈 때 있다.
  1-2. 투교: 수비 라인업 `refresh`
2. 1구 단위 row 기록할 때 : 수비 라인업 조회하고 기록

# 4. 로드 이후 순서

1. `textOrder`로 중계 텍스트를 나눈다.
2. `seqno` 순서로 텍스트를 로드, 파싱한다.
3. 파싱 결과에 따라 game status를 변경
4. 텍스트 전체 파싱, 진행 끝나면 csv로 저장

# 파싱 순서

1. 타석 단위로 스택 처리, `type` 따라 달라짐
    - `type`에 따라서...
        - `0` : 이닝 status refresh
        - `1` : 투구 결과 및 데이터 기록
        - `2` : 교체 반영 -> 배팅 오더, 수비 라인업
        - `7` : 시스템 메시지
            - continue
        - `8` : x번타자/대타 xxx -> game status에서 현재 타자 이름 변경
        - `13` : __타자주자__ 타석 결과(득점 없는 타석)
            - 타석 stack 생성
            - 타석 결과 나왔다는 flag True로 바꾸기
            - base 결과, out 결과, score 결과 stack에 쌓는다
            - **곧바로 출력하지 않는다**: PA 끝까지 stack에 쌓아서 주루 결과까치 처리 후 출력
            - 결과도 즉각 game status에 반영하지 않는다            
        - `14` : __주자__ 주루/아웃(득점 없는 타석)
            - 타석 결과 flag 확인 : False인 경우 새로 stack 생성
                - 타석 도중에 나올 수 있기 때문(도루 성공&실패, 견제사, 보크, 폭투 등)
            - stack 다 쌓은 다음에 처리
        - `23` : __타자주자__ 타석 결과(득점 있는 타석)
            - `13`과 동일, '홈인' 메시지는 득점 반영 / 주자 베이스 처리
        - `24` : __주자__ 주루/아웃(득점 있는 타석)
            - `14`와 동일, '홈인' 메시지는 득점 반영 / 주자 베이스 처리
        - `44` : 파울 에러 (pass)
            - continue
        - `99` : 경기 종료 메시지
            - continue
2. 스택 처리
    - print stack을 만들고, 여기에 출력 row를 쌓아둠
    - pitch 들어오면 현재 game status 출력해 row 적립, game status 수정
    - 주자 스택은 모아서 처리
        - 중간에 나오는 주자 스택은 모아서 따로 row로 출력
            - row 하나로 출력할 수 있도록 처리
        - 마지막에 나오는 스택은 타석 결과가 있으면 함께 처리
            - 타석 결과 없을 수도 있음(견제사 3아웃 등)
    - 13, 14, 23, 24 들어오면 tail row의 description 수정(append), game status 수정
    - 13, 14, 23, 24에서 주자 처리는 다음 순서로
        - 13, 14, 23, 24 들어오면 current runner로 설정
        - while current row type in (13, 14, 23, 24)
          - if current runner = None
              - current runner <= row runner
              - current runner의 play 처리, location 변경
          - if row runner != current runner
              - current runner의 description과 after play location을 확정
              - current runner <= row runner
              - current runner의 play 처리, location 변경

# 주루 경우의 수 정리

before | after | 출력
-|-|-
. . . | O . . | 1루타; 볼넷; 실책으로 출루; 몸에 맞는 볼; 낫아웃 폭투; 낫아웃 포일; 내야안타; 번트안타; 타격방해; 야수선택;
. . . | . O . | 2루타; 1루타/내야안타/번트안타/낫아웃 폭투/낫아웃 포일 + 실책으로 출루
. . . | . . O | 3루타; 1루타/2루타/내야안타/번트안타/낫아웃 폭투/낫아웃 포일 + 실책으로 출루
. . . | . . . | 

# 타석 결과, 주자 주루 stack 처리

1. 타석 마지막 나오는 `13`, `14`, `23`, `24`
    - 타석 결과 있는 경우 : `13`, `23`
        1. 끝까지 row 스태킹
        2. 타석 결과 row : 결과 획득
        3. 타석 결과 row : 주루 처리
        4. (option) 주루 결과 row : 주루 처리
        5. 반복
    - 타석 결과 없는 경우 : `14`, `24`
        1. 끝까지 row 스태킹
        2. 주루 결과 row : 주루 처리 append append
        3. 반복 후 출력
2. 타석 중간에 나오는 `14`, `24`
    1. while로 투구 전까지 결과 스태킹
    2. 결과 처리

# 타석 결과 종류 정리

keyword | 간단 분류 | 상세 분류
-|-|-
삼진 | 삼진 | 삼진
볼넷 | 볼넷 | 볼넷
자동 고의 4구 | 고의 4구 | 고의 4구
고의4구 | 고의 4구 | 고의 4구
몸에 | 몸에 맞는 공 | 몸에 맞는 공
1루타 | 안타 | 안타
내야안타 | 내야 안타 | 내야 안타
번트안타 | 번트 안타 | 번트 안타
안타 | 안타 | 안타
2루타 | 2루타 | 2루타
3루타 | 3루타 | 3루타
홈런 | 홈런 | 홈런
낫아웃 폭투 | 낫아웃 출루 | 낫아웃 폭투
낫아웃 포일 | 낫아웃 출루 | 낫아웃 포일
낫 아웃 | 삼진 | 낫아웃 삼진
낫아웃 다른주자 수비 실책 | 낫아웃 출루 | 낫아웃 다른 주자 수비 실책
낫아웃 다른주자 수비 | 낫아웃 출루 | 낫아웃 다른 주자 포스 아웃
땅볼로 출루 | 포스 아웃 | 땅볼 아웃
땅볼 아웃 | 포스 아웃 | 땅볼 아웃
플라이 아웃 | 필드 아웃 | 플라이 아웃
인필드 | 필드 아웃 | 인필드 플라이
파울플라이 | 필드 아웃 | 파울 플라이 아웃
라인드라이브 아웃 | 필드 아웃 | 라인드라이브 아웃
번트 아웃 | 필드 아웃 | 번트 아웃
병살타 | 병살타 | 병살타
희생번트 아웃 | 희생 번트 | 희생 번트
희생플라이 아웃 | 희생 플라이 | 희생 플라이
희생플라이아웃 | 희생 플라이 | 희생 플라이
쓰리번트 | 삼진 | 쓰리번트 삼진
타구맞음 | 필드 아웃 | 타구맞음 아웃
희생번트 실책 | 실책 | 실책
희생번트 야수선택 | 야수 선택 | 야수 선택
야수선택 | 야수 선택 | 야수 선택
실책 | 실책 | 실책
타격방해 | 타격 방해 | 타격 방해
삼중살 | 삼중살 | 삼중살
부정타격 | 필드 아웃 | 부정 타격 아웃
번트 | 번트 안타 | 안타

# 타석 결과 예외
1. 병살타
    - 선행주자 2명 포스아웃 : 병살타가 아닌 '땅볼로 출루'로 기록.
        - [예시](https://sports.news.naver.com/gameCenter/textRelay.nhn?gameId=20090705HTHH0&category=kbo) https://sports.news.naver.com/gameCenter/textRelay.nhn?gameId=20090705HTHH0&category=kbo
        
        

`type`
- `0` : x회초(말) xx 공격
- `1` : 투구 (x구 ...)
- `2` : 교체
- `7` : 시스템 메시지 (비디오 판독, ...)
- `8` : x번타자 xxx
- `13` : 타자주자 타석 결과(득점 없는 타석)
- `14` : 주자 주루/아웃(득점 없는 타석)
- `23` : 타자주자 타석 결과(득점 있는 타석)
- `24` : 주자 주루/아웃(득점 있는 타석)
- `44` : 파울 에러 (pass)
- `99` : 경기 종료 메시지

`pitchResult`
- `T` : s __T__ rike 스트라이크
- `F` : __F__oul 파울
- `S` : __S__wing 헛스윙
- `B` : __B__all 볼
- `H` : __H__it 타격

- `pitchNum` : 문자중계 텍스트(`textOptionList`)에 포함되는 투구수, 정합성 확인 안됨, 신뢰하지 말 것
- `ballcount` : pitch data(`ptsOptionList`)에 포함되는 투구수, `ptsPitchId` 오류 있을 때는 1개씩 밀리는 문제, 신뢰하지 말 것
- `inn` : 문자중계와 pitch data에 모두 포함, 지금은 Pitch data에 있는걸 가져오는 중. 신뢰하지 말 것

1. 스트라이크/볼/헛스윙/파울 : 바로 저장
    - 1-1. 몸에맞는 공: pa_result 확인 후 수정
2. 타격 : 딜레이 저장
  - description, pa_result, pa_result_detail 한꺼번에 저장